<a href="https://colab.research.google.com/github/olinyoder2534/TensorflowPractice/blob/main/ImageClassification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import cv2
import pandas as pd
import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
ImageShape = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=ImageShape+(3,))
])

In [4]:
dog = Image.open('/content/IMG_2331.JPG').resize(ImageShape)

In [5]:
dog = dog.rotate(270)
#dog

In [6]:
dog = np.array(dog)/255.0

In [7]:
result = classifier.predict(dog[np.newaxis, ...])
result

1/1 [==============================] - 1s 950ms/step


array([[ 0.31111565, -1.9600838 , -0.41632408, ..., -2.1500363 ,
        -1.9272399 ,  0.7957212 ]], dtype=float32)

In [8]:
result.shape

(1, 1001)

In [9]:
np.argmax(result)

731

In [10]:
image_labels = []

file_path = '/content/ImageNetLabels.txt'

with open(file_path, "r") as f:
    image_labels = f.read().splitlines()
image_labels[731]

#not correct, probably because the image contains more of a stick than a dog

'plow'

In [11]:
sorted_indices = np.argsort(result[0])[::-1]
top_ten = sorted_indices[:10]
top_ten = top_ten.tolist()
top_ten

[731, 492, 793, 622, 429, 209, 871, 307, 235, 841]

In [12]:
with open(file_path, "r") as f:
    image_labels = f.read().splitlines()

for index in top_ten:
    print(image_labels[index])

plow
chain saw
shovel
lawn mower
barrow
Labrador retriever
tricycle
rhinoceros beetle
Rottweiler
swab


Flowers Dataset

In [13]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [14]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [15]:
#create dictionaries for flower types
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [16]:
#cleaning - resizing and setting X and y
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

X = np.array(X)
y = np.array(y)

In [17]:
X.shape

(3670, 224, 224, 3)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [19]:
X_train = X_train/255.0
X_test = X_test/255.0

In [20]:
#model creation
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [21]:
num_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_flowers)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
#not enough RAM to run this lol
model.compile(
  optimizer="adam",
  loss = 'sparse_categorical_crossentropy',
  metrics = ['acc'])

model.fit(X_train, y_train, epochs=5)

In [ ]:
model.evaluate(X_test,y_test)